# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          31 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/30 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.10053


extracting tarball to tmp_2210.10053...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.10059


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.10053/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2210.10707...

 done.


Retrieving document from  https://arxiv.org/e-print/2210.10736


extracting tarball to tmp_2210.10736...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10707-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10707) | **X-ray galaxies selected from HyperLEDA database**  |
|| <mark>N. Pulatova</mark>, et al. -- incl., <mark>R. Seeburger</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *11 pages and 5 figures*|
|**Abstract**| We cross-matched the 4XMM-DR10 catalog with the HyperLEDA database and obtained the new sample of galaxies that contain X-ray sources. Excluding duplicate observations and false matches, we present a total of 7759 galaxies with X-ray sources. In the current work, we present general properties of the sample: namely the distribution in equatorial coordinates, radial velocity distribution, morphological type, and X-ray fluxes. The sample includes morphological classification for 5241 galaxies with X-ray emission, almost half of which, 42\%, are elliptical (E, E-S0). Most galaxies in the sample have nuclear X-ray emission (6313 or 81\%), and the remaining 1443 (19\%) present X-ray emission from the host galaxy. This sample can be used for future deep studies of multi wavelengths properties of the galaxies with X-ray emission. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10053-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10053) | **Nozzle Shocks, Disk Tearing and Streamers Drive Rapid Accretion in 3D  GRMHD Simulations of Warped Thin Disks**  |
|| Nicholas Kaaz, et al. -- incl., <mark>Matthew T.P. Liska</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *17 pages, 10 figures. Submitted to ApJ. Comments welcome and encouraged!*|
|**Abstract**| The angular momentum of gas feeding a black hole (BH) is typically misaligned with respect to the BH spin, resulting in a tilted accretion disk. Rotation of the BH drags the surrounding space-time, manifesting as Lense-Thirring torques that lead to disk precession and warping. We study these processes by simulating a thin ($H/r=0.02$), highly tilted ($\mathcal{T}=65^\circ$) accretion disk around a rapidly rotating ($a=0.9375$) BH at extremely high resolutions, which we performed using the general-relativistic magnetohydrodynamic (GRMHD) code H-AMR. The disk becomes significantly warped and continuously tears into two individually precessing sub-disks. We find that mass accretion rates far exceed the standard $\alpha$-viscosity expectations. We identify two novel dissipation mechanisms specific to warped disks that are the main drivers of accretion, distinct from the local turbulent stresses that are usually thought to drive accretion. In particular, we identify extreme scale height oscillations that occur twice an orbit throughout our disk. When the scale height compresses, `nozzle' shocks form, dissipating orbital energy and driving accretion. Separate from this phenomenon, there is also extreme dissipation at the location of the tear. This leads to the formation of low-angular momentum `streamers' that rain down onto the inner sub-disk, shocking it. The addition of low angular momentum gas to the inner sub-disk causes it to rapidly accrete, even when it is transiently aligned with the BH spin and thus unwarped. These mechanisms, if general, significantly modify the standard accretion paradigm. Additionally, they may drive structural changes on much shorter timescales than expected in $\alpha$-disks, potentially explaining some of the extreme variability observed in active galactic nuclei. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10059-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10059) | **The MillenniumTNG Project: High-precision predictions for matter  clustering and halo statistics**  |
|| <mark>César Hernández-Aguayo</mark>, et al. -- incl., <mark>Lars Hernquist</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *submitted to MNRAS, 23 pages, 19 figures, for future public data release, see this https URL*|
|**Abstract**| Cosmological inference with large galaxy surveys requires theoretical models that combine precise predictions for large-scale structure with robust and flexible galaxy formation modelling throughout a sufficiently large cosmic volume. Here, we introduce the MillenniumTNG (MTNG) project which combines the hydrodynamical galaxy formation model of IllustrisTNG with the large volume of the Millennium simulation. Our largest hydrodynamic simulation, covering (500 Mpc/h)^3 = (740 Mpc)^3, is complemented by a suite of dark-matter-only simulations with up to 4320^3 dark matter particles (a mass resolution of 1.32 x 10^8 Msun/h) using the fixed-and-paired technique to reduce large-scale cosmic variance. The hydro simulation adds 4320^3 gas cells, achieving a baryonic mass resolution of 2 x 10^7 Msun/h. High time-resolution merger trees and direct lightcone outputs facilitate the construction of a new generation of semi-analytic galaxy formation models that can be calibrated against both the hydro simulation and observation, and then applied to even larger volumes - MTNG includes a flagship simulation with 1.1 trillion dark matter particles and massive neutrinos in a volume of (3000 Mpc)^3. In this introductory analysis we carry out convergence tests on basic measures of non-linear clustering such as the matter power spectrum, the halo mass function and halo clustering, and we compare simulation predictions to those from current cosmological emulators. We also use our simulations to study matter and halo statistics, such as halo bias and clustering at the baryonic acoustic oscillation scale. Finally we measure the impact of baryonic physics on the matter and halo distributions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10699-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10699) | **Hubble WFC3 Spectroscopy of the Rocky Planet L 98-59 b: No Evidence for  a Cloud-Free Primordial Atmosphere**  |
|| <mark>Li Zhou</mark>, Bo Ma, Yonghao Wang, Yinan Zhu |
|*Appeared on*| *2022-10-20*|
|*Comments*| *15 pages, 11 figures, 6 tables, accepted for publication in AJ*|
|**Abstract**| We are using archived data from HST of transiting exoplanet L~98-59~b to place constraints on its potentially hot atmosphere. We analyze the data from five transit visits and extract the final combined transmission spectrum using Iraclis. Then we use the inverse atmospheric retrieval code TauREx to analyze the combined transmission spectrum. There is a weak absorption feature near 1.40~$\mu m$ and 1.55~$\mu m$ in the transmission spectrum, which can be modeled by a cloudy atmosphere with abundant HCN. However, the unrealistically high abundance of HCN derived cannot be explained by any equilibrium chemical model with reasonable assumptions. Thus, the likeliest scenario is that L~98-59~b has a flat, featureless transmission spectrum in the WFC3/G141 bandpass due to a thin atmosphere with high mean molecular weight, an atmosphere with an opaque aerosol layer, or no atmosphere, and it is very unlikely for L~98-59~b to have a clear hydrogen-dominated primary atmosphere. Due to the narrow wavelength coverage and low spectral resolution of HST/WFC3 G141 grism observation, we cannot tell these different scenarios apart. Our simulation shows future higher precision measurements over wider wavelengths from the James Webb Space Telescope (JWST) can be used to better characterize the planetary atmosphere of L~98-59~b. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10736-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10736) | **Reconnection-Driven Energy Cascade in Magnetohydrodynamic Turbulence**  |
|| Chuanfei Dong, et al. -- incl., <mark>Liang Wang</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *32 pages, 8 figures, the world's largest 3D MHD turbulence simulation using a fifth-order scheme*|
|**Abstract**| Magnetohydrodynamic turbulence regulates the transfer of energy from large to small scales in many astrophysical systems, including the solar atmosphere. We perform three-dimensional magnetohydrodynamic simulations with unprecedentedly large magnetic Reynolds number to reveal how rapid reconnection of magnetic field lines changes the classical paradigm of the turbulent energy cascade. By breaking elongated current sheets into chains of small magnetic flux ropes (or plasmoids), magnetic reconnection leads to a new range of turbulent energy cascade, where the rate of energy transfer is controlled by the growth rate of the plasmoids. As a consequence, the turbulent energy spectra steepen and attain a spectral index of -2.2 that is accompanied by changes in the anisotropy of turbulence eddies. The omnipresence of plasmoids and their consequences on, e.g., solar coronal heating, can be further explored with current and future spacecraft and telescopes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10102-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10102) | **Why is zero spatial curvature special?**  |
|| Raul Jimenez, et al. -- incl., <mark>Daniel F. Litim</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| **|
|**Abstract**| Evidence for almost spatial flatness of the Universe has been provided from several observational probes, including the Cosmic Microwave Background (CMB) and Baryon Acoustic Oscillations (BAO) from galaxy clustering data. However, other than inflation, and in this case only in the limit of infinite time, there is no strong a priori motivation for a spatially flat Universe. Using the renormalization group (RG) technique in curved spacetime, we present in this work a theoretical motivation for spatial flatness. Starting from a general spacetime, the first step of the RG, coarse-graining, gives a Friedmann-Lema\^itre-Robertson-Walker (FLRW) metric with a set of parameters. Then, we study the rescaling properties of the curvature parameter, and find that zero spatial curvature of the FLRW metric is singled out as the unique scale-free, non-singular background for cosmological perturbations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error invalid header</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10112-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10112) | **Euclid Near Infrared Spectrometer and Photometer instrument flight model  presentation, performance and ground calibration results summary**  |
|| T. Maciaszek, et al. -- incl., <mark>K. Jahnke</mark>, <mark>E. Franceschi</mark>, <mark>S. Ligori</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *18 pages, to appear in Proceedings of the SPIE*|
|**Abstract**| The NISP (Near Infrared Spectrometer and Photometer) is one of the two Euclid instruments. It operates in the near-IR spectral region (950-2020nm) as a photometer and spectrometer. The instrument is composed of: a cold (135 K) optomechanical subsystem consisting of a Silicon carbide structure, an optical assembly, a filter wheel mechanism, a grism wheel mechanism, a calibration unit, and a thermal control system, a detection system based on a mosaic of 16 H2RG with their front-end readout electronic, and a warm electronic system (290 K) composed of a data processing / detector control unit and of an instrument control unit that interfaces with the spacecraft via a 1553 bus for command and control and via Spacewire links for science data. This paper presents: the final architecture of the flight model instrument and subsystems, and the performance and the ground calibration measurement done at NISP level and at Euclid Payload Module level at operational cold temperature. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.10402-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.10402) | **Solar Ring Mission: Building a Panorama of the Sun and Inner-heliosphere**  |
|| Yuming Wang, et al. -- incl., <mark>Linjie Chen</mark>, <mark>Linhua Deng</mark>, <mark>Li Feng</mark>, <mark>Jingnan Guo</mark>, <mark>Yang Guo</mark>, <mark>Jiansen He</mark>, <mark>Xiaolei Li</mark>, <mark>Yiren Li</mark>, <mark>Jiajia Liu</mark>, <mark>Kai Liu</mark>, <mark>Liu Liu</mark>, <mark>Rui Liu</mark>, <mark>Rui Liu</mark>, <mark>Lidong Xia</mark>, <mark>Jinglan Xie</mark>, <mark>Shenyi Zhang</mark>, <mark>Quanhao Zhang</mark>, <mark>Yonghe Zhang</mark>, <mark>Jinsong Zhao</mark> |
|*Appeared on*| *2022-10-20*|
|*Comments*| *41 pages, 6 figures, 1 table, to be published in Advances in Space Research*|
|**Abstract**| Solar Ring (SOR) is a proposed space science mission to monitor and study the Sun and inner heliosphere from a full 360{\deg} perspective in the ecliptic plane. It will deploy three 120{\deg}-separated spacecraft on the 1-AU orbit. The first spacecraft, S1, locates 30{\deg} upstream of the Earth, the second, S2, 90{\deg} downstream, and the third, S3, completes the configuration. This design with necessary science instruments, e.g., the Doppler-velocity and vector magnetic field imager, wide-angle coronagraph, and in-situ instruments, will allow us to establish many unprecedented capabilities: (1) provide simultaneous Doppler-velocity observations of the whole solar surface to understand the deep interior, (2) provide vector magnetograms of the whole photosphere - the inner boundary of the solar atmosphere and heliosphere, (3) provide the information of the whole lifetime evolution of solar featured structures, and (4) provide the whole view of solar transients and space weather in the inner heliosphere. With these capabilities, Solar Ring mission aims to address outstanding questions about the origin of solar cycle, the origin of solar eruptions and the origin of extreme space weather events. The successful accomplishment of the mission will construct a panorama of the Sun and inner-heliosphere, and therefore advance our understanding of the star and the space environment that holds our life. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.10707.md
    + _build/html/tmp_2210.10707/./HyperLEDA-X-B-che.jpg
    + _build/html/tmp_2210.10707/./RA-Dec.png
    + _build/html/tmp_2210.10707/./Angular_separation.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# X-ray galaxies selected from HyperLEDA database

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.10707-b31b1b.svg)](https://arxiv.org/abs/2210.10707)<mark>Appeared on: 2022-10-20</mark> - _11 pages and 5 figures_

</div>
<div id="authors">

<mark><mark>N. Pulatova</mark></mark>, et al. -- incl., <mark><mark>R. Seeburger</mark></mark>

</div>
<div id="abstract">

**Abstract:** We cross-matched the 4XMM-DR10 catalog with the HyperLEDA database and obtained the new sample of galaxies that contain X-ray sources. Excluding duplicate observations and false matches, we present a total of 7759 galaxies with X-ray sources. In the current work, we present general properties of the sample: namely the distribution in equatorial coordinates, radial velocity distribution, morphological type, and X-ray fluxes. The sample includes morphological classification for 5241 galaxies with X-ray emission, almost half of which, 42\%, are elliptical (E, E-S0). Most galaxies in the sample have nuclear X-ray emission (6313 or 81\%), and the remaining 1443 (19\%) present X-ray emission from the host galaxy. This sample can be used for future deep studies of multi wavelengths properties of the galaxies with X-ray emission.\\[1ex]{\bf Key words:}X-rays: galaxies, catalogues, galaxies: statistics

</div>

<div id="div_fig1">

<img src="tmp_2210.10707/./HyperLEDA-X-B-che.jpg" alt="Fig4" width="100%"/>

**Figure 4. -** Optical B magnitude and X-ray flux. The three regions marked with the numbers 1-3 corresponds to: 1 - brightest galaxies with low redshift; 2 - the limit of  XMM sensitivity; 3- the gap between two populations of active galaxies. (*fig4*)

</div>
<div id="div_fig2">

<img src="tmp_2210.10707/./RA-Dec.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution in equatorial coordinates (*fig1*)

</div>
<div id="div_fig3">

<img src="tmp_2210.10707/./Angular_separation.png" alt="Fig2" width="100%"/>

**Figure 2. -** The angular separation between the position of objects
in HyperLEDA and XMM-Newton catalogues, in $arcsec (")$. (*fig2*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

69  publications in the last 7 days.
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers